In [123]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [99]:
data = pd.read_csv('./data/train.csv')
#data['Age'] = data['Age'].apply(pd.to_numeric, errors='coerce')
data = data.dropna()

In [100]:
def check_adult_child(age):
    if age > 18:
        return 'A'
    else:
        return 'C'

In [60]:
#data['Adult/child'] = data['Age'].apply(check_adult_child)

In [101]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


### We can compare the survival rates for each column to determine the relative importance of a feature in the survival of a passenger.

In [102]:
data = data[['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']]
training_data = data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']]
training_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
1,1,1,female,38.0,1,0,C
3,1,1,female,35.0,1,0,S
6,0,1,male,54.0,0,0,S
10,1,3,female,4.0,1,1,S
11,1,1,female,58.0,0,0,S


In [103]:
def compare(group, data):
    return data.groupby([group])['Survived'].sum()*100/data.groupby([group])['Survived'].count()

In [104]:
compare('Pclass', data)

Pclass
1    67.088608
2    80.000000
3    50.000000
Name: Survived, dtype: float64

In [105]:
compare('Sex', data)

Sex
female    93.181818
male      43.157895
Name: Survived, dtype: float64

In [108]:
compare("Embarked", data)

Embarked
C    73.846154
Q    50.000000
S    63.793103
Name: Survived, dtype: float64

In [109]:
compare('SibSp', data)

SibSp
0    62.727273
1    73.437500
2    83.333333
3    66.666667
Name: Survived, dtype: float64

In [110]:
compare('Parch', data)

Parch
0    65.573770
1    70.270270
2    73.913043
4     0.000000
Name: Survived, dtype: float64

### Convert any categorical data to numeric 

In [111]:
def cat_to_num(series):
    series = series.astype('category')
    return series.cat.codes   

In [112]:
training_data['Gender'] = data[['Sex']].apply(cat_to_num)
training_data['Port'] = data[['Embarked']].apply(cat_to_num)
training_data = training_data[['Survived', 'Pclass', 'Gender', 'Age', 'SibSp', 'Parch', 'Port']]
train, test = train_test_split(training_data, test_size=0.2)
train.head()
#training_data.head()
#len(training_data)

,Survived,Pclass,Gender,Age,SibSp,Parch,Port
102,0,1,1,21.0,0,1,2
449,1,1,1,52.0,0,0,2
136,1,1,0,19.0,0,2,2
862,1,1,0,48.0,0,0,2
689,1,1,0,15.0,0,1,2


### Set up an instance of the classifier and feed training data

In [124]:
classifier = DecisionTreeClassifier(max_leaf_nodes=25)

In [125]:
classifier = classifier.fit(train[['Pclass','Gender', 'Age', 'SibSp', 'Parch', 'Port']], train['Survived'])

In [126]:
classifier

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=25, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [127]:
classifier.feature_importances_

array([ 0.00998583,  0.37231069,  0.50274433,  0.05478552,  0.04087809,
        0.01929553])

In [135]:
with open('./out/titanic.dot', 'w') as f:
    f = tree.export_graphviz(classifier, feature_names=['Pclass', 'Gender', 'Age', 'SibSp', 'Parch', 'Port'], out_file=f)